In [1]:
%load_ext autoreload
%autoreload 2
import os
import json
from tqdm import tqdm
import torch 
import torch.nn as nn
import models
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import Adam
from torchvision import datasets, transforms

print(f"device avaible : {'cuda' if torch.cuda.is_available() else 'cpu'}")

device avaible : cuda


# **Dataset**

Création de deux datasets distincts : 
- A contient les images des classes de 0 à 4
- B contient les images des classes de 5 à 9 

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


train_set, val_set = train_test_split(list(range(len(cifar10))), test_size=0.2, random_state=42)

def create_subset(dataset, indices, class_range):
    subset_indices = [i for i in indices if dataset.targets[i] in class_range]
    return Subset(dataset, subset_indices)

train_set_A = create_subset(cifar10, train_set, range(0, 5))
train_set_B = create_subset(cifar10, train_set, range(5, 10))

val_set_A = create_subset(cifar10, val_set, range(0, 5))
val_set_B = create_subset(cifar10, val_set, range(5, 10))


train_loader_A = DataLoader(train_set_A, batch_size=64, shuffle=True)
train_loader_B = DataLoader(train_set_B, batch_size=64, shuffle=True)

val_loader_A = DataLoader(val_set_A, batch_size=64, shuffle=False)
val_loader_B = DataLoader(val_set_B, batch_size=64, shuffle=False)

test_loader_A = DataLoader(create_subset(cifar10_test, range(len(cifar10_test)), range(0, 5)), batch_size=64, shuffle=False)
test_loader_B = DataLoader(create_subset(cifar10_test, range(len(cifar10_test)), range(5, 10)), batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
loss_fn = nn.CrossEntropyLoss()
path = "./resultats_cifar/"

resA  = []
resB = []
print("Entrainement du modèle A")
print()
for _ in range(5): 
    modelA = models.CNN_cifar()
    modelA.to(device)
    optimizerA = Adam(modelA.parameters())
    modelA.train_model(train_loader_A,test_loader_A, loss_fn, n_epochs=10, optimizer=optimizerA)
    resA.append(modelA.test(test_loader_A))
torch.save(modelA, "./models_cifar/baseA.torch")

print()
print("Entrainement du modèle B")
print()
for _ in range(5): 
    modelB = models.CNN_cifar()
    optimizerB= Adam(modelB.parameters())
    modelB.train_model(train_loader_B,test_loader_B, loss_fn, n_epochs=10, optimizer=optimizerB)
    resB.append(modelB.test(test_loader_B))
torch.save(modelB, "./models_cifar/baseB.torch")

with open(os.path.join(path, "base.json"), 'a') as file:
        json.dump({"model A" : resA}, file)
        json.dump({"model B" : resB}, file)

Entrainement du modèle A

using device cuda
Epoch 1, Loss: 1.2493
Test set: Avg. loss: 1.0153, Accuracy: 2956/5000 (59.12%)
Epoch 2, Loss: 0.9512
Test set: Avg. loss: 0.8920, Accuracy: 3189/5000 (63.78%)
Epoch 3, Loss: 0.8612
Test set: Avg. loss: 0.8687, Accuracy: 3285/5000 (65.70%)
Epoch 4, Loss: 0.7778
Test set: Avg. loss: 0.8122, Accuracy: 3411/5000 (68.22%)
Epoch 5, Loss: 0.6839
Test set: Avg. loss: 0.8288, Accuracy: 3373/5000 (67.46%)
Epoch 6, Loss: 0.5778
Test set: Avg. loss: 0.8147, Accuracy: 3472/5000 (69.44%)
Epoch 7, Loss: 0.4331
Test set: Avg. loss: 0.9086, Accuracy: 3424/5000 (68.48%)
Epoch 8, Loss: 0.2817
Test set: Avg. loss: 1.1422, Accuracy: 3399/5000 (67.98%)
Epoch 9, Loss: 0.1762
Test set: Avg. loss: 1.2862, Accuracy: 3375/5000 (67.50%)
Epoch 10, Loss: 0.1163
Test set: Avg. loss: 1.6133, Accuracy: 3363/5000 (67.26%)
Test set: Avg. loss: 1.6133, Accuracy: 3363/5000 (67.26%)
using device cuda
Epoch 1, Loss: 1.2301
Test set: Avg. loss: 1.0229, Accuracy: 2842/5000 (56.84%)

# Modèle BNB

In [6]:
path = "./resultats_cifar/"


basemodel = torch.load("./models_cifar/baseB.torch")
loss_fn = nn.CrossEntropyLoss()


final = {}
for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(5):
        newmodel = models.CNN_cifar()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        for param in newmodel.model[:j].parameters():  
            param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=10, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "B",
        "couche copiees": j,
        "couche reentrainees": 8 - j,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "BNB.json"), 'w') as file:
    json.dump(final, file)

/tmp/ipykernel_4439/599675842.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models_cifar/baseB.torch")
Etape :   0%|          | 0/8 [00:00<?,

using device cuda
Epoch 1, Loss: 1.1393
Test set: Avg. loss: 0.9826, Accuracy: 3091/5000 (61.82%)
Epoch 2, Loss: 0.8266
Test set: Avg. loss: 0.7388, Accuracy: 3606/5000 (72.12%)
Epoch 3, Loss: 0.7159
Test set: Avg. loss: 0.7150, Accuracy: 3683/5000 (73.66%)
Epoch 4, Loss: 0.6194
Test set: Avg. loss: 0.6502, Accuracy: 3779/5000 (75.58%)
Epoch 5, Loss: 0.5229
Test set: Avg. loss: 0.6415, Accuracy: 3854/5000 (77.08%)
Epoch 6, Loss: 0.4311
Test set: Avg. loss: 0.6274, Accuracy: 3910/5000 (78.20%)
Epoch 7, Loss: 0.3060
Test set: Avg. loss: 0.6719, Accuracy: 3870/5000 (77.40%)
Epoch 8, Loss: 0.2025
Test set: Avg. loss: 0.7753, Accuracy: 3823/5000 (76.46%)
Epoch 9, Loss: 0.1245
Test set: Avg. loss: 0.9542, Accuracy: 3881/5000 (77.62%)
Epoch 10, Loss: 0.0760
Test set: Avg. loss: 1.0424, Accuracy: 3806/5000 (76.12%)
Test set: Avg. loss: 1.0424, Accuracy: 3806/5000 (76.12%)
using device cuda
Epoch 1, Loss: 1.1711
Test set: Avg. loss: 0.9143, Accuracy: 3126/5000 (62.52%)
Epoch 2, Loss: 0.8240
Tes

Etape :  12%|█▎        | 1/8 [05:37<39:25, 337.95s/it]

Test set: Avg. loss: 1.1623, Accuracy: 3730/5000 (74.60%)
using device cuda
Epoch 1, Loss: 1.1452
Test set: Avg. loss: 0.8504, Accuracy: 3353/5000 (67.06%)
Epoch 2, Loss: 0.7804
Test set: Avg. loss: 0.7615, Accuracy: 3567/5000 (71.34%)
Epoch 3, Loss: 0.6707
Test set: Avg. loss: 0.6942, Accuracy: 3710/5000 (74.20%)
Epoch 4, Loss: 0.5716
Test set: Avg. loss: 0.6058, Accuracy: 3881/5000 (77.62%)
Epoch 5, Loss: 0.4822
Test set: Avg. loss: 0.6340, Accuracy: 3892/5000 (77.84%)
Epoch 6, Loss: 0.3902
Test set: Avg. loss: 0.6730, Accuracy: 3796/5000 (75.92%)
Epoch 7, Loss: 0.2851
Test set: Avg. loss: 0.7527, Accuracy: 3852/5000 (77.04%)
Epoch 8, Loss: 0.1842
Test set: Avg. loss: 0.9253, Accuracy: 3801/5000 (76.02%)
Epoch 9, Loss: 0.1365
Test set: Avg. loss: 1.1868, Accuracy: 3767/5000 (75.34%)
Epoch 10, Loss: 0.1088
Test set: Avg. loss: 1.2260, Accuracy: 3745/5000 (74.90%)
Test set: Avg. loss: 1.2260, Accuracy: 3745/5000 (74.90%)
using device cuda
Epoch 1, Loss: 1.2093
Test set: Avg. loss: 0.91

Etape :  25%|██▌       | 2/8 [11:16<33:48, 338.10s/it]

...

Test set: Avg. loss: 1.5000, Accuracy: 3574/5000 (71.48%)
using device cuda
Epoch 1, Loss: 1.1045
Test set: Avg. loss: 0.8843, Accuracy: 3272/5000 (65.44%)
Epoch 2, Loss: 0.8618
Test set: Avg. loss: 0.8565, Accuracy: 3340/5000 (66.80%)
Epoch 3, Loss: 0.7648
Test set: Avg. loss: 0.7684, Accuracy: 3574/5000 (71.48%)
Epoch 4, Loss: 0.6539
Test set: Avg. loss: 0.7360, Accuracy: 3646/5000 (72.92%)
Epoch 5, Loss: 0.5472
Test set: Avg. loss: 0.7477, Accuracy: 3690/5000 (73.80%)
Epoch 6, Loss: 0.4171
Test set: Avg. loss: 0.8552, Accuracy: 3611/5000 (72.22%)
Epoch 7, Loss: 0.2816
Test set: Avg. loss: 1.0909, Accuracy: 3566/5000 (71.32%)
Epoch 8, Loss: 0.1974
Test set: Avg. loss: 1.0793, Accuracy: 3649/5000 (72.98%)
Epoch 9, Loss: 0.1380
Test set: Avg. loss: 1.2872, Accuracy: 3601/5000 (72.02%)
Epoch 10, Loss: 0.0932
Test set: Avg. loss: 1.4969, Accuracy: 3589/5000 (71.78%)
Test set: Avg. loss: 1.4969, Accuracy: 3589/5000 (71.78%)
using device cuda
Epoch 1, Loss: 1.1495
Test set: Avg. loss: 0.94

Etape :  88%|████████▊ | 7/8 [38:56<05:31, 331.77s/it]

Test set: Avg. loss: 1.4424, Accuracy: 3579/5000 (71.58%)
using device cuda
Epoch 1, Loss: 1.1515
Test set: Avg. loss: 1.0206, Accuracy: 3031/5000 (60.62%)
Epoch 2, Loss: 0.8908
Test set: Avg. loss: 0.9725, Accuracy: 3163/5000 (63.26%)
Epoch 3, Loss: 0.7789
Test set: Avg. loss: 0.8598, Accuracy: 3403/5000 (68.06%)
Epoch 4, Loss: 0.6660
Test set: Avg. loss: 0.8609, Accuracy: 3395/5000 (67.90%)
Epoch 5, Loss: 0.5438
Test set: Avg. loss: 0.9429, Accuracy: 3378/5000 (67.56%)
Epoch 6, Loss: 0.3966
Test set: Avg. loss: 1.0887, Accuracy: 3337/5000 (66.74%)
Epoch 7, Loss: 0.2774
Test set: Avg. loss: 1.2595, Accuracy: 3351/5000 (67.02%)
Epoch 8, Loss: 0.1889
Test set: Avg. loss: 1.3892, Accuracy: 3336/5000 (66.72%)
Epoch 9, Loss: 0.1335
Test set: Avg. loss: 1.6608, Accuracy: 3364/5000 (67.28%)
Epoch 10, Loss: 0.1116
Test set: Avg. loss: 1.7163, Accuracy: 3398/5000 (67.96%)
Test set: Avg. loss: 1.7163, Accuracy: 3398/5000 (67.96%)
using device cuda
Epoch 1, Loss: 1.0601
Test set: Avg. loss: 0.88

Etape : 100%|██████████| 8/8 [44:21<00:00, 332.70s/it]

Test set: Avg. loss: 1.6070, Accuracy: 3474/5000 (69.48%)


# Modèle ANB

In [8]:
path = "./resultats_cifar/"


basemodel = torch.load("./models_cifar/baseA.torch")
loss_fn = nn.CrossEntropyLoss()

final = {}
for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(5):
        newmodel = models.CNN_cifar()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        for param in newmodel.model[:j].parameters():  
            param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=10, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "A",
        "couche copiees": j,
        "couche reentrainees": 8 - j,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "ANB.json"), 'w') as file:
    json.dump(final, file)

/tmp/ipykernel_4439/4281061378.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models_cifar/baseA.torch")
Etape :   0%|          | 0/8 [00:00<?

using device cuda
Epoch 1, Loss: 1.1137
Test set: Avg. loss: 0.8570, Accuracy: 3341/5000 (66.82%)
Epoch 2, Loss: 0.7823
Test set: Avg. loss: 0.7191, Accuracy: 3658/5000 (73.16%)
Epoch 3, Loss: 0.6549
Test set: Avg. loss: 0.6952, Accuracy: 3702/5000 (74.04%)
Epoch 4, Loss: 0.5735
Test set: Avg. loss: 0.6373, Accuracy: 3831/5000 (76.62%)
Epoch 5, Loss: 0.4699
Test set: Avg. loss: 0.6566, Accuracy: 3823/5000 (76.46%)
Epoch 6, Loss: 0.3502
Test set: Avg. loss: 0.7217, Accuracy: 3827/5000 (76.54%)
Epoch 7, Loss: 0.2260
Test set: Avg. loss: 0.8533, Accuracy: 3848/5000 (76.96%)
Epoch 8, Loss: 0.1342
Test set: Avg. loss: 0.9840, Accuracy: 3879/5000 (77.58%)
Epoch 9, Loss: 0.0888
Test set: Avg. loss: 1.2355, Accuracy: 3817/5000 (76.34%)
Epoch 10, Loss: 0.0743
Test set: Avg. loss: 1.1942, Accuracy: 3850/5000 (77.00%)
Test set: Avg. loss: 1.1942, Accuracy: 3850/5000 (77.00%)
using device cuda
Epoch 1, Loss: 1.1407
Test set: Avg. loss: 0.9237, Accuracy: 3203/5000 (64.06%)
Epoch 2, Loss: 0.8434
Tes

Etape :  12%|█▎        | 1/8 [05:35<39:07, 335.34s/it]

Test set: Avg. loss: 1.2709, Accuracy: 3754/5000 (75.08%)
using device cuda
Epoch 1, Loss: 1.1775
Test set: Avg. loss: 0.9027, Accuracy: 3245/5000 (64.90%)
Epoch 2, Loss: 0.8458
Test set: Avg. loss: 0.7596, Accuracy: 3567/5000 (71.34%)
Epoch 3, Loss: 0.6953
Test set: Avg. loss: 0.6632, Accuracy: 3767/5000 (75.34%)
Epoch 4, Loss: 0.5887
Test set: Avg. loss: 0.6773, Accuracy: 3789/5000 (75.78%)
Epoch 5, Loss: 0.4845
Test set: Avg. loss: 0.5872, Accuracy: 3921/5000 (78.42%)
Epoch 6, Loss: 0.3843
Test set: Avg. loss: 0.6563, Accuracy: 3936/5000 (78.72%)
Epoch 7, Loss: 0.2671
Test set: Avg. loss: 0.7124, Accuracy: 3942/5000 (78.84%)
Epoch 8, Loss: 0.1718
Test set: Avg. loss: 0.7490, Accuracy: 3917/5000 (78.34%)
Epoch 9, Loss: 0.1115
Test set: Avg. loss: 0.8494, Accuracy: 3953/5000 (79.06%)
Epoch 10, Loss: 0.0831
Test set: Avg. loss: 0.9640, Accuracy: 3931/5000 (78.62%)
Test set: Avg. loss: 0.9640, Accuracy: 3931/5000 (78.62%)
using device cuda
Epoch 1, Loss: 1.1021
Test set: Avg. loss: 0.86

Etape :  25%|██▌       | 2/8 [11:10<33:32, 335.35s/it]

...

Test set: Avg. loss: 1.2823, Accuracy: 3568/5000 (71.36%)
using device cuda
Epoch 1, Loss: 1.1122
Test set: Avg. loss: 0.9168, Accuracy: 3185/5000 (63.70%)
Epoch 2, Loss: 0.8719
Test set: Avg. loss: 0.8634, Accuracy: 3308/5000 (66.16%)
Epoch 3, Loss: 0.7632
Test set: Avg. loss: 0.8565, Accuracy: 3416/5000 (68.32%)
Epoch 4, Loss: 0.6772
Test set: Avg. loss: 0.9294, Accuracy: 3352/5000 (67.04%)
Epoch 5, Loss: 0.5785
Test set: Avg. loss: 0.7576, Accuracy: 3634/5000 (72.68%)
Epoch 6, Loss: 0.4578
Test set: Avg. loss: 0.7993, Accuracy: 3642/5000 (72.84%)
Epoch 7, Loss: 0.3324
Test set: Avg. loss: 0.8740, Accuracy: 3610/5000 (72.20%)
Epoch 8, Loss: 0.2025
Test set: Avg. loss: 1.1331, Accuracy: 3528/5000 (70.56%)
Epoch 9, Loss: 0.1436
Test set: Avg. loss: 1.2188, Accuracy: 3586/5000 (71.72%)
Epoch 10, Loss: 0.0933
Test set: Avg. loss: 1.6257, Accuracy: 3514/5000 (70.28%)
Test set: Avg. loss: 1.6257, Accuracy: 3514/5000 (70.28%)
using device cuda
Epoch 1, Loss: 1.1567
Test set: Avg. loss: 0.95

Etape :  88%|████████▊ | 7/8 [38:33<05:28, 328.24s/it]

Test set: Avg. loss: 1.3828, Accuracy: 3492/5000 (69.84%)
using device cuda
Epoch 1, Loss: 1.0827
Test set: Avg. loss: 0.9438, Accuracy: 3116/5000 (62.32%)
Epoch 2, Loss: 0.8877
Test set: Avg. loss: 0.8931, Accuracy: 3259/5000 (65.18%)
Epoch 3, Loss: 0.7751
Test set: Avg. loss: 0.8236, Accuracy: 3441/5000 (68.82%)
Epoch 4, Loss: 0.6495
Test set: Avg. loss: 0.8398, Accuracy: 3441/5000 (68.82%)
Epoch 5, Loss: 0.5063
Test set: Avg. loss: 0.9318, Accuracy: 3444/5000 (68.88%)
Epoch 6, Loss: 0.3385
Test set: Avg. loss: 1.0563, Accuracy: 3420/5000 (68.40%)
Epoch 7, Loss: 0.2045
Test set: Avg. loss: 1.2149, Accuracy: 3397/5000 (67.94%)
Epoch 8, Loss: 0.1238
Test set: Avg. loss: 1.5019, Accuracy: 3437/5000 (68.74%)
Epoch 9, Loss: 0.0929
Test set: Avg. loss: 1.5416, Accuracy: 3452/5000 (69.04%)
Epoch 10, Loss: 0.0745
Test set: Avg. loss: 1.7409, Accuracy: 3453/5000 (69.06%)
Test set: Avg. loss: 1.7409, Accuracy: 3453/5000 (69.06%)
using device cuda
Epoch 1, Loss: 1.1250
Test set: Avg. loss: 0.94

Etape : 100%|██████████| 8/8 [43:56<00:00, 329.55s/it]

Test set: Avg. loss: 1.5244, Accuracy: 3436/5000 (68.72%)


# Modèle BNB+

In [12]:
path = "./resultats_cifar/"


basemodel = torch.load("./models_cifar/baseB.torch")
loss_fn = nn.CrossEntropyLoss()

final = {}

for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(5):
        newmodel = models.CNN_cifar()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        # for param in newmodel.conv[:j].parameters():  
        #     param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=10, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "B",
        "couche copiees": j,
        "couche reentrainees": 8,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "BNB+.json"), 'w') as file:
    json.dump(final, file)

/tmp/ipykernel_4439/1066040676.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models_cifar/baseB.torch")
Etape :   0%|          | 0/8 [00:00<?

using device cuda
Epoch 1, Loss: 1.2141
Test set: Avg. loss: 0.9469, Accuracy: 3157/5000 (63.14%)
Epoch 2, Loss: 0.8451
Test set: Avg. loss: 0.7795, Accuracy: 3523/5000 (70.46%)
Epoch 3, Loss: 0.6978
Test set: Avg. loss: 0.6802, Accuracy: 3716/5000 (74.32%)
Epoch 4, Loss: 0.6080
Test set: Avg. loss: 0.6198, Accuracy: 3883/5000 (77.66%)
Epoch 5, Loss: 0.5118
Test set: Avg. loss: 0.5761, Accuracy: 3960/5000 (79.20%)
Epoch 6, Loss: 0.4186
Test set: Avg. loss: 0.5795, Accuracy: 3963/5000 (79.26%)
Epoch 7, Loss: 0.3255
Test set: Avg. loss: 0.6894, Accuracy: 3909/5000 (78.18%)
Epoch 8, Loss: 0.2117
Test set: Avg. loss: 0.7786, Accuracy: 3963/5000 (79.26%)
Epoch 9, Loss: 0.1383
Test set: Avg. loss: 0.9077, Accuracy: 3918/5000 (78.36%)
Epoch 10, Loss: 0.1002
Test set: Avg. loss: 0.9333, Accuracy: 3971/5000 (79.42%)
Test set: Avg. loss: 0.9333, Accuracy: 3971/5000 (79.42%)
using device cuda
Epoch 1, Loss: 1.2316
Test set: Avg. loss: 0.9502, Accuracy: 3157/5000 (63.14%)
Epoch 2, Loss: 0.8301
Tes

Etape :  12%|█▎        | 1/8 [05:34<39:03, 334.79s/it]

Test set: Avg. loss: 1.0161, Accuracy: 3936/5000 (78.72%)
using device cuda
Epoch 1, Loss: 1.2830
Test set: Avg. loss: 1.0015, Accuracy: 3009/5000 (60.18%)
Epoch 2, Loss: 0.8989
Test set: Avg. loss: 0.8333, Accuracy: 3406/5000 (68.12%)
Epoch 3, Loss: 0.7304
Test set: Avg. loss: 0.7246, Accuracy: 3622/5000 (72.44%)
Epoch 4, Loss: 0.6336
Test set: Avg. loss: 0.6414, Accuracy: 3791/5000 (75.82%)
Epoch 5, Loss: 0.5559
Test set: Avg. loss: 0.6579, Accuracy: 3765/5000 (75.30%)
Epoch 6, Loss: 0.4828
Test set: Avg. loss: 0.6473, Accuracy: 3877/5000 (77.54%)
Epoch 7, Loss: 0.3834
Test set: Avg. loss: 0.6651, Accuracy: 3883/5000 (77.66%)
Epoch 8, Loss: 0.2869
Test set: Avg. loss: 0.7404, Accuracy: 3880/5000 (77.60%)
Epoch 9, Loss: 0.1975
Test set: Avg. loss: 0.8620, Accuracy: 3821/5000 (76.42%)
Epoch 10, Loss: 0.1474
Test set: Avg. loss: 0.9207, Accuracy: 3889/5000 (77.78%)
Test set: Avg. loss: 0.9207, Accuracy: 3889/5000 (77.78%)
using device cuda
Epoch 1, Loss: 1.2219
Test set: Avg. loss: 0.89

Etape :  25%|██▌       | 2/8 [11:10<33:31, 335.33s/it]

...

Test set: Avg. loss: 0.8074, Accuracy: 4049/5000 (80.98%)
using device cuda
Epoch 1, Loss: 0.9070
Test set: Avg. loss: 0.7246, Accuracy: 3670/5000 (73.40%)
Epoch 2, Loss: 0.6191
Test set: Avg. loss: 0.6088, Accuracy: 3890/5000 (77.80%)
Epoch 3, Loss: 0.5139
Test set: Avg. loss: 0.5341, Accuracy: 4002/5000 (80.04%)
Epoch 4, Loss: 0.4249
Test set: Avg. loss: 0.5479, Accuracy: 4061/5000 (81.22%)
Epoch 5, Loss: 0.3459
Test set: Avg. loss: 0.5403, Accuracy: 4081/5000 (81.62%)
Epoch 6, Loss: 0.2439
Test set: Avg. loss: 0.6389, Accuracy: 4041/5000 (80.82%)
Epoch 7, Loss: 0.1672
Test set: Avg. loss: 0.6797, Accuracy: 4096/5000 (81.92%)
Epoch 8, Loss: 0.1223
Test set: Avg. loss: 0.8268, Accuracy: 4007/5000 (80.14%)
Epoch 9, Loss: 0.0876
Test set: Avg. loss: 0.9053, Accuracy: 3980/5000 (79.60%)
Epoch 10, Loss: 0.0721
Test set: Avg. loss: 0.8684, Accuracy: 4079/5000 (81.58%)
Test set: Avg. loss: 0.8684, Accuracy: 4079/5000 (81.58%)
using device cuda
Epoch 1, Loss: 0.9203
Test set: Avg. loss: 0.68

Etape :  88%|████████▊ | 7/8 [39:07<05:35, 335.61s/it]

Test set: Avg. loss: 0.9330, Accuracy: 4044/5000 (80.88%)
using device cuda
Epoch 1, Loss: 0.8385
Test set: Avg. loss: 0.6210, Accuracy: 3857/5000 (77.14%)
Epoch 2, Loss: 0.5759
Test set: Avg. loss: 0.5322, Accuracy: 4018/5000 (80.36%)
Epoch 3, Loss: 0.4554
Test set: Avg. loss: 0.5175, Accuracy: 4045/5000 (80.90%)
Epoch 4, Loss: 0.3842
Test set: Avg. loss: 0.5549, Accuracy: 4051/5000 (81.02%)
Epoch 5, Loss: 0.2997
Test set: Avg. loss: 0.5582, Accuracy: 4077/5000 (81.54%)
Epoch 6, Loss: 0.2005
Test set: Avg. loss: 0.6129, Accuracy: 4109/5000 (82.18%)
Epoch 7, Loss: 0.1358
Test set: Avg. loss: 0.6349, Accuracy: 4073/5000 (81.46%)
Epoch 8, Loss: 0.1132
Test set: Avg. loss: 0.8291, Accuracy: 4077/5000 (81.54%)
Epoch 9, Loss: 0.0868
Test set: Avg. loss: 0.8125, Accuracy: 4047/5000 (80.94%)
Epoch 10, Loss: 0.0711
Test set: Avg. loss: 0.8400, Accuracy: 4082/5000 (81.64%)
Test set: Avg. loss: 0.8400, Accuracy: 4082/5000 (81.64%)
using device cuda
Epoch 1, Loss: 0.8053
Test set: Avg. loss: 0.70

Etape : 100%|██████████| 8/8 [44:43<00:00, 335.46s/it]

Test set: Avg. loss: 0.8391, Accuracy: 4089/5000 (81.78%)


# Modèle ANB+

In [11]:
path = "./resultats_cifar/"


basemodel = torch.load("./models_cifar/baseA.torch")
loss_fn = nn.CrossEntropyLoss()


for j in tqdm(range(1, 9), desc="Etape "):
    l = []
    for _ in range(5):
        newmodel = models.CNN_cifar()
        for i in range(1,j): # copie de certaines couches
            newmodel.model[i] = basemodel.model[i]
        # for param in newmodel.conv[:j].parameters():  
        #     param.requires_grad = False

        optimizer = Adam(filter(lambda p: p.requires_grad, newmodel.parameters()))
        newmodel.train_model(train_loader_B, test_loader_B, loss_fn, n_epochs=10, optimizer=optimizer)
        l.append(newmodel.test(test_loader_B))
   
    res = {
        "model base": "A",
        "couche copiees": j,
        "couche reentrainees": 8,
        "resultats": l
    }
    final[j] = res
with open(os.path.join(path, "ANB+.json"), 'a') as file:
    json.dump(final, file)

/tmp/ipykernel_4439/3236697046.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  basemodel = torch.load("./models_cifar/baseA.torch")
Etape :   0%|          | 0/8 [00:00<?

using device cuda
Epoch 1, Loss: 1.1605
Test set: Avg. loss: 0.9140, Accuracy: 3155/5000 (63.10%)
Epoch 2, Loss: 0.8436
Test set: Avg. loss: 0.8316, Accuracy: 3418/5000 (68.36%)
Epoch 3, Loss: 0.6931
Test set: Avg. loss: 0.6862, Accuracy: 3682/5000 (73.64%)
Epoch 4, Loss: 0.5971
Test set: Avg. loss: 0.6908, Accuracy: 3725/5000 (74.50%)
Epoch 5, Loss: 0.5194
Test set: Avg. loss: 0.6188, Accuracy: 3835/5000 (76.70%)
Epoch 6, Loss: 0.4361
Test set: Avg. loss: 0.6618, Accuracy: 3825/5000 (76.50%)
Epoch 7, Loss: 0.3489
Test set: Avg. loss: 0.6925, Accuracy: 3881/5000 (77.62%)
Epoch 8, Loss: 0.2626
Test set: Avg. loss: 0.7211, Accuracy: 3912/5000 (78.24%)
Epoch 9, Loss: 0.1952
Test set: Avg. loss: 0.8812, Accuracy: 3818/5000 (76.36%)
Epoch 10, Loss: 0.1443
Test set: Avg. loss: 1.0363, Accuracy: 3858/5000 (77.16%)
Test set: Avg. loss: 1.0363, Accuracy: 3858/5000 (77.16%)
using device cuda
Epoch 1, Loss: 1.1304
Test set: Avg. loss: 0.9188, Accuracy: 3276/5000 (65.52%)
Epoch 2, Loss: 0.7958
Tes

Etape :  12%|█▎        | 1/8 [05:37<39:25, 337.90s/it]

Test set: Avg. loss: 0.8923, Accuracy: 3953/5000 (79.06%)
using device cuda
Epoch 1, Loss: 1.1430
Test set: Avg. loss: 0.9105, Accuracy: 3218/5000 (64.36%)
Epoch 2, Loss: 0.8177
Test set: Avg. loss: 0.7653, Accuracy: 3486/5000 (69.72%)
Epoch 3, Loss: 0.6676
Test set: Avg. loss: 0.6656, Accuracy: 3726/5000 (74.52%)
Epoch 4, Loss: 0.5612
Test set: Avg. loss: 0.6872, Accuracy: 3791/5000 (75.82%)
Epoch 5, Loss: 0.4682
Test set: Avg. loss: 0.6222, Accuracy: 3919/5000 (78.38%)
Epoch 6, Loss: 0.3643
Test set: Avg. loss: 0.6599, Accuracy: 3907/5000 (78.14%)
Epoch 7, Loss: 0.2474
Test set: Avg. loss: 0.7634, Accuracy: 3857/5000 (77.14%)
Epoch 8, Loss: 0.1666
Test set: Avg. loss: 0.8305, Accuracy: 3929/5000 (78.58%)
Epoch 9, Loss: 0.1020
Test set: Avg. loss: 1.0048, Accuracy: 3913/5000 (78.26%)
Epoch 10, Loss: 0.0830
Test set: Avg. loss: 1.0165, Accuracy: 3894/5000 (77.88%)
Test set: Avg. loss: 1.0165, Accuracy: 3894/5000 (77.88%)
using device cuda
Epoch 1, Loss: 1.1428
Test set: Avg. loss: 0.84

Etape :  25%|██▌       | 2/8 [11:16<33:51, 338.56s/it]

...

Test set: Avg. loss: 0.8921, Accuracy: 4044/5000 (80.88%)
using device cuda
Epoch 1, Loss: 0.8928
Test set: Avg. loss: 0.6758, Accuracy: 3755/5000 (75.10%)
Epoch 2, Loss: 0.6167
Test set: Avg. loss: 0.5663, Accuracy: 3975/5000 (79.50%)
Epoch 3, Loss: 0.5054
Test set: Avg. loss: 0.5277, Accuracy: 4031/5000 (80.62%)
Epoch 4, Loss: 0.4248
Test set: Avg. loss: 0.5458, Accuracy: 4073/5000 (81.46%)
Epoch 5, Loss: 0.3483
Test set: Avg. loss: 0.5983, Accuracy: 3974/5000 (79.48%)
Epoch 6, Loss: 0.2520
Test set: Avg. loss: 0.6145, Accuracy: 4056/5000 (81.12%)
Epoch 7, Loss: 0.1770
Test set: Avg. loss: 0.6756, Accuracy: 4035/5000 (80.70%)
Epoch 8, Loss: 0.1164
Test set: Avg. loss: 0.8543, Accuracy: 3943/5000 (78.86%)
Epoch 9, Loss: 0.1036
Test set: Avg. loss: 0.8157, Accuracy: 4069/5000 (81.38%)
Epoch 10, Loss: 0.0819
Test set: Avg. loss: 0.8533, Accuracy: 4002/5000 (80.04%)
Test set: Avg. loss: 0.8533, Accuracy: 4002/5000 (80.04%)
using device cuda
Epoch 1, Loss: 0.8951
Test set: Avg. loss: 0.67

Etape :  88%|████████▊ | 7/8 [39:31<05:39, 339.22s/it]

Test set: Avg. loss: 0.8482, Accuracy: 4114/5000 (82.28%)
using device cuda
Epoch 1, Loss: 0.8314
Test set: Avg. loss: 0.6677, Accuracy: 3792/5000 (75.84%)
Epoch 2, Loss: 0.5657
Test set: Avg. loss: 0.5593, Accuracy: 3997/5000 (79.94%)
Epoch 3, Loss: 0.4507
Test set: Avg. loss: 0.4983, Accuracy: 4084/5000 (81.68%)
Epoch 4, Loss: 0.3608
Test set: Avg. loss: 0.5341, Accuracy: 4104/5000 (82.08%)
Epoch 5, Loss: 0.2462
Test set: Avg. loss: 0.5957, Accuracy: 4094/5000 (81.88%)
Epoch 6, Loss: 0.1512
Test set: Avg. loss: 0.7245, Accuracy: 4026/5000 (80.52%)
Epoch 7, Loss: 0.1099
Test set: Avg. loss: 0.7873, Accuracy: 4051/5000 (81.02%)
Epoch 8, Loss: 0.0768
Test set: Avg. loss: 0.7327, Accuracy: 4106/5000 (82.12%)
Epoch 9, Loss: 0.0523
Test set: Avg. loss: 1.1067, Accuracy: 4010/5000 (80.20%)
Epoch 10, Loss: 0.0560
Test set: Avg. loss: 1.1026, Accuracy: 3943/5000 (78.86%)
Test set: Avg. loss: 1.1026, Accuracy: 3943/5000 (78.86%)
using device cuda
Epoch 1, Loss: 0.8148
Test set: Avg. loss: 0.60

Etape : 100%|██████████| 8/8 [45:08<00:00, 338.52s/it]

Test set: Avg. loss: 0.8445, Accuracy: 4110/5000 (82.20%)
